# Useful Utilities

**Authors:** Olivia Lynn

**Last Run Successfully:** September 20, 2023

This is a notebook that contains various utilities that may be used when working with RAIL.

## Setting Things Up

In [ ]:
import rail

### Listing imported stages (1/2)

Let's list out our currently imported stages. Right now, this will only be what we get by importing `rail` and `rail.stages`.

In [ ]:
import rail.stages
for val in rail.core.RailStage.pipeline_stages.values():
    print(val[0])

### Import and attach all

Using `rail.stages.import_and_attach_all()` lets you import all packages within the RAIL ecosystem at once. 

This kind of blanket import is a useful shortcut; however, it will be slower than specific imports, as you will import things you'll never need. 

As of such, `import_and_attach_all` is recommended for new users and those who wish to do rapid exploration with notebooks; pipelines designed to be run at scale would generally prefer lightweight, specific imports.


In [ ]:
import rail
import rail.stages
rail.stages.import_and_attach_all()


Now that we've attached all available stages to rail.stages, we can use `from rail.stages import *` to let us omit prefixes. 

To see this in action:

In [ ]:
# with prefix

print(rail.core.util_stages.ColumnMapper)

In [ ]:
# without prefix

try:
    print(ColumnMapper)
except Exception as e: 
    print(e)

In [ ]:
from rail.stages import *

In [ ]:
print(ColumnMapper)

### Listing imported stages (2/2)

Now, let's try listing imported stages again.

Note that we can now just call `RailStage` instead of `rail.core.RailStage`.

In [ ]:
for val in RailStage.pipeline_stages.values():
    print(val[0])

We can use this list of imported stages to browse for specifics, such as looking through our available estimators.

**Note:** this will only filter through what you've imported, so if you haven't imported everything above, this will not be a complete list of all estimators available in RAIL.

In [ ]:
for val in RailStage.pipeline_stages.values():
    if issubclass(val[0], rail.estimation.estimator.CatEstimator):
        print(val[0])

### Listing keys in the Data Store (1/2)

Let's list out the keys in the Data Store to see what data we have stored.

First, we must set up the Data Store:

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

And because we've only just created the store, as you may have guessed, it is empty. 

We'll come back to this in a bit.

In [ ]:
DS.keys()

### Finding data files with find_rail_file

We need to define our flow file that we'll use in our pipeline

If we already know its path, we can just point directly to the file (relative to the directory that holds our `rail/` directory):

In [ ]:
from rail.core.utils import RAILDIR

flow_file = os.path.join(
    RAILDIR, "rail/examples_data/goldenspike_data/data/pretrained_flow.pkl"
)

But if we aren't sure where our file is (or we're just feeling lazy) we can use `find_rail_file`.

This is especially helpful in cases where our installation is spread out, and some rail modules are located separately from others.

In [ ]:
from rail.core.utils import find_rail_file

flow_file = find_rail_file('examples_data/goldenspike_data/data/pretrained_flow.pkl')

We can set our FLOWDIR based on the location of our flow file, too.

In [ ]:
os.environ['FLOWDIR'] = os.path.dirname(flow_file)

In [ ]:
# Now, we have to set up some other variables for our pipeline:

bands = ["u", "g", "r", "i", "z", "y"]
band_dict = {band: f"mag_{band}_lsst" for band in bands}
rename_dict = {f"mag_{band}_lsst_err": f"mag_err_{band}_lsst" for band in bands}
post_grid = [float(x) for x in np.linspace(0.0, 5, 21)]


## Creating the Pipeline

In [ ]:
import ceci

In [ ]:
# Make some stages

flow_engine_test = FlowCreator.make_stage(
    name="flow_engine_test", model=flow_file, n_samples=50
)
col_remapper_test = ColumnMapper.make_stage(
    name="col_remapper_test", hdf5_groupname="", columns=rename_dict
)
#flow_engine_test.sample(6, seed=0).data

In [ ]:
# Add the stages to the pipeline

pipe = ceci.Pipeline.interactive()
stages = [flow_engine_test, col_remapper_test]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
# Connect stages

col_remapper_test.connect_input(flow_engine_test)

## Introspecting the Pipeline

### Listing keys in the Data Store (2/2)

Now that we have a some data in the Data Store, let's take another look at it.

In [ ]:
DS.keys()

### Getting names of stages in the pipeline

In [ ]:
pipe.stage_names

### Getting the configuration of a particular stage

Let's take a look a the config of the first stage we just listed above.

In [ ]:
pipe.flow_engine_test.config

### Updating a configuration value
 
We can update config values even after the stage has been created. Let's give it a try.

In [ ]:
pipe.flow_engine_test.config.update(seed=42)

pipe.flow_engine_test.config

### Listing stage outputs (as both tags and aliased tags)

Let's get the list of outputs as 'tags'.

These are how the stage thinks of the outputs, as a list names associated to DataHandle types.

In [ ]:
pipe.flow_engine_test.outputs

We can also get the list of outputs as 'aliased tags'.

These are how the pipeline thinks of the outputs, as a unique key that points to a particular file

In [ ]:
pipe.flow_engine_test._outputs

### Listing all pipeline methods and parameters that can be set

If you'd like to take a closer look at what you can do with a pipeline, use `dir(pipe)` to list out available methods and parameters.

In [ ]:
for item in dir(pipe):
    if '__' not in item:
        print(item)

## Initializing the Pipeline

### Toggling resume mode

We can turn 'resume mode' on when initializing a pipeline.

Resume mode lets us skip stages that already have output files, so we don't have to rerun the same stages as we iterate on a pipeline.

Just add a `resume=True` to do so.

In [ ]:
pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=True), None
)

Running `pipe.stages` should show order of classes, or all the stages this pipeline will run.

In [ ]:
pipe.stages

## Managing notebooks with git

_(thank you to https://stackoverflow.com/a/58004619)_

You can modify your git settings to run a filter over certain files before they are added to git. This will leave the original file on disk as-is, but commit the "cleaned" version.


First, add the following to your local `.git/config` file (or global `~/.gitconfig`):

Then, create a `.gitattributes` file in your directory with notebooks and add the following line: